In [1]:
# Import libraries and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import keras
import tensorflow as tf
import warnings
import os
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
warnings.filterwarnings('ignore')

In [2]:
url = 'Datasets/2012_2019_PlatteRiverWeir_features_merged_all.csv'

dfCNN = pd.read_csv(url)

In [3]:
dfCNN

,SensorTime,CaptureTime,Filename,Agency,SiteNumber,TimeZone,Stage,Discharge,CalcTimestamp,width,...,WeirPt2X,WeirPt2Y,WwRawLineMin,WwRawLineMax,WwRawLineMean,WwRawLineSigma,WwCurveLineMin,WwCurveLineMax,WwCurveLineMean,WwCurveLineSigma
0,2012-06-09 13:15:00,2012-06-09T13:09:07,StateLineWeir_20120609_Farrell_001.jpg,USGS,6674500,MDT,2.99,916.0,2020-03-11T16:58:28,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,2012-06-09 13:15:00,2012-06-09T13:10:29,StateLineWeir_20120609_Farrell_002.jpg,USGS,6674500,MDT,2.99,916.0,2020-03-11T16:58:33,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,2012-06-09 13:45:00,2012-06-09T13:44:01,StateLineWeir_20120609_Farrell_003.jpg,USGS,6674500,MDT,2.96,873.0,2020-03-11T16:58:40,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
3,2012-06-09 14:45:00,2012-06-09T14:44:30,StateLineWeir_20120609_Farrell_004.jpg,USGS,6674500,MDT,2.94,846.0,2020-03-11T16:58:47,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,2012-06-09 15:45:00,2012-06-09T15:44:59,StateLineWeir_20120609_Farrell_005.jpg,USGS,6674500,MDT,2.94,846.0,2020-03-11T16:58:55,4288,...,-1,-1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42054,2019-10-11 09:00:00,2019-10-11T08:59:53,StateLineWeir_20191011_Farrell_409.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:53,4288,...,2446,1900,9284.0,77521.0,38385.370066,15952.029728,0.0,70085.0,37550.894823,16444.401209
42055,2019-10-11 10:00:00,2019-10-11T09:59:52,StateLineWeir_20191011_Farrell_410.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:54,4288,...,2440,1900,10092.0,74614.0,40162.989292,15467.708856,0.0,70061.0,39397.339095,16009.008049
42056,2019-10-11 11:00:00,2019-10-11T10:59:52,StateLineWeir_20191011_Farrell_411.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:55,4288,...,2447,1900,7067.0,83260.0,42095.946590,16770.357949,0.0,76335.0,41350.006568,17489.374617
42057,2019-10-11 12:00:00,2019-10-11T11:59:53,StateLineWeir_20191011_Farrell_412.jpg,USGS,6674500,MDT,2.54,434.0,2020-03-12T16:23:56,4288,...,2443,1900,6283.0,83045.0,45345.490954,17498.432849,0.0,78882.0,44553.920296,18268.294896


In [30]:
input_path_train = []

for filename in os.listdir('train_test_val_images/train/'):
    for path in 'train_test_val_images/train/images/'+dfCNN[' Filename']:
        input_path_train.append(os.path.join('train_test_val_images/train/images/', filename, path))

In [31]:
df_train = pd.DataFrame()
df_train['images'] = input_path_train
df_train['filename'] = dfCNN[' Filename']
df_train['discharge'] = dfCNN[' Discharge']
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,images,filename,discharge
0,train_test_val_images/train/images/images\trai...,StateLineWeir_20180402_Farrell_175.jpg,220.00
1,train_test_val_images/train/images/images\trai...,StateLineWeir_20141014_Farrell_642.JPG,416.00
2,train_test_val_images/train/images/images\trai...,StateLineWeir_20141022_Farrell_804.JPG,359.00
3,train_test_val_images/train/images/images\trai...,StateLineWeir_20170129_Farrell_088.jpg,301.00
4,train_test_val_images/train/images/images\trai...,StateLineWeir_20150414_Farrell_897.JPG,155.00
...,...,...,...
42054,train_test_val_images/train/images/images\trai...,StateLineWeir_20130612_Farrell_367.JPG,7.05
42055,train_test_val_images/train/images/images\trai...,StateLineWeir_20150729_Farrell_242.JPG,1160.00
42056,train_test_val_images/train/images/images\trai...,StateLineWeir_20171220_Farrell_103.jpg,209.00
42057,train_test_val_images/train/images/images\trai...,StateLineWeir_20160122_Farrell_598.JPG,255.00


In [27]:
input_path_test = []

for filename in os.listdir('train_test_val_images/test'):
    for path in 'train_test_val_images/test/images/'+dfCNN[' Filename']:
        input_path_test.append(os.path.join('train_test_val_images/test/images', filename, path))

In [28]:
df_test = pd.DataFrame()
df_test['images'] = input_path_test
df_test['filename'] = dfCNN[' Filename']
df_test['discharge'] = dfCNN[' Discharge']
df_test = df_test.sample(frac=1).reset_index(drop=True)
df_test

,images,filename,discharge
0,train_test_val_images/test/images\images\train...,StateLineWeir_20180911_Farrell_032.jpg,990.0
1,train_test_val_images/test/images\images\train...,StateLineWeir_20160730_Farrell_954.jpg,1750.0
2,train_test_val_images/test/images\images\train...,StateLineWeir_20140204_Farrell_949.JPG,0.0
3,train_test_val_images/test/images\images\train...,StateLineWeir_20170114_Farrell_671.jpg,293.0
4,train_test_val_images/test/images\images\train...,StateLineWeir_20130727_Farrell_882.JPG,1210.0
...,...,...,...
42054,train_test_val_images/test/images\images\train...,StateLineWeir_20170227_Farrell_878.jpg,340.0
42055,train_test_val_images/test/images\images\train...,StateLineWeir_20121027_Farrell_983.JPG,155.0
42056,train_test_val_images/test/images\images\train...,StateLineWeir_20140427_Farrell_020.JPG,159.0
42057,train_test_val_images/test/images\images\train...,StateLineWeir_20170613_Farrell_829.jpg,3430.0


In [25]:
input_path_val = []

for filename in os.listdir('train_test_val_images/val/'):
    for path in 'train_test_val_images/val/images/'+dfCNN[' Filename']:
        input_path_val.append(os.path.join('train_test_val_images/val/images/', filename, path))

In [26]:
df_val = pd.DataFrame()
df_val['images'] = input_path_val
df_val['filename'] = dfCNN[' Filename']
df_val['discharge'] = dfCNN[' Discharge']
df_val = df_val.sample(frac=1).reset_index(drop=True)
df_val

,images,filename,discharge
0,train_test_val_images/val/images/images\train_...,StateLineWeir_20120620_Farrell_250.jpg,1270.0
1,train_test_val_images/val/images/images\train_...,StateLineWeir_20180807_Farrell_985.jpg,1240.0
2,train_test_val_images/val/images/images\train_...,StateLineWeir_20170409_Farrell_015.jpg,614.0
3,train_test_val_images/val/images/images\train_...,StateLineWeir_20150511_Farrell_503.JPG,704.0
4,train_test_val_images/val/images/images\train_...,StateLineWeir_20121128_Farrell_940.JPG,231.0
...,...,...,...
42054,train_test_val_images/val/images/images\train_...,StateLineWeir_20130213_Farrell_010.JPG,164.0
42055,train_test_val_images/val/images/images\train_...,StateLineWeir_20140313_Farrell_309.JPG,155.0
42056,train_test_val_images/val/images/images\train_...,StateLineWeir_20150511_Farrell_502.JPG,704.0
42057,train_test_val_images/val/images/images\train_...,StateLineWeir_20160415_Farrell_454.JPG,287.0


In [32]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu',input_shape=(512,512,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='linear'))

In [33]:
model.compile(optimizer='adam', metrics=['mse'], loss=[tf.keras.metrics.MeanSquaredError()])

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 510, 510, 64)      1792      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 255, 255, 64)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 253, 253, 128)     73856     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 126, 126, 128)    0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 124, 124, 256)     295168    
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 62, 62, 256)     

In [13]:
df_train.columns

Index(['images', 'filename', 'discharge'], dtype='object')

In [22]:
df_val.columns

Index(['images', 'filename', 'discharge'], dtype='object')

In [35]:
datagen = ImageDataGenerator()
train_data = datagen.flow_from_dataframe(
    df_train,
    # directory='train_test_val_images/train/',
    x_col='filename',
    y_col='discharge',
    target_size=(512,512),
    batch_size=8,
    class_mode='raw',
    validate_filenames=True
)

val_data = datagen.flow_from_dataframe(
    df_val,
    # directory='train_test_val_images/val/',
    x_col='filename',
    y_col='discharge',
    target_size=(512,512),
    batch_size=8,
    class_mode='raw',
    validate_filenames=True
)

# train_data = datagen.flow_from_directory(
#     directory='train_test_val_images/train/',
#     target_size=(512,512),
#     batch_size=256,
#     class_mode='input'
# )
#
# val_data = datagen.flow_from_directory(
#     directory='train_test_val_images/val/',
#     target_size=(512,512),
#     batch_size=256,
#     class_mode='input'
# )

Found 0 validated image filenames.
Found 0 validated image filenames.


In [19]:
history = model.fit(train_data, epochs=5, validation_data=val_data)

ValueError: Asked to retrieve element 0, but the Sequence has length 0